# Kmeans over a set of GeoTiffs

This notebook loads a set of GeoTiffs into a **RDD** of Tiles, with each Tile being a band in the GeoTiff. Each GeoTiff file contains **SpringIndex-** or **LastFreeze-** value for one year over the entire USA.

Kmeans takes years as dimensions. Hence, the matrix has cells as rows and the years as columns. To cluster on all years, the matrix needs to be transposed. The notebook has two flavors of matrix transpose, locally by the Spark-driver or distributed using the Spark-workers. Once transposed the matrix is converted to a **RDD** of dense vectors to be used by **Kmeans** algorithm from **Spark-MLlib**. The end result is a grid where each cell has a cluster ID which is then saved into a SingleBand GeoTiff. By saving the result into a GeoTiff, the reader can plot it using a Python notebook as the one defined in the [python examples](../examples/python).

<span style="color:red">In this notebook the reader only needs to modify the variables in **Mode of Operation Setup**</span>.

## Dependencies

In [1]:
import sys.process._

import java.io.{ByteArrayInputStream, ByteArrayOutputStream, ObjectInputStream, ObjectOutputStream}


import geotrellis.proj4.CRS
import geotrellis.raster.{CellType, ArrayTile, DoubleArrayTile, Tile, UByteCellType}
import geotrellis.raster.io.geotiff._
import geotrellis.raster.io.geotiff.writer.GeoTiffWriter
import geotrellis.raster.io.geotiff.{GeoTiff, SinglebandGeoTiff}
import geotrellis.spark.io.hadoop._
import org.apache.hadoop.io._
import geotrellis.vector.{Extent, ProjectedExtent}
import org.apache.spark.broadcast.Broadcast
import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
import org.apache.spark.mllib.linalg.distributed.{CoordinateMatrix, MatrixEntry, RowMatrix}
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}

import org.apache.hadoop.io.{IOUtils, SequenceFile}
import org.apache.hadoop.io.SequenceFile.Writer

//Spire is a numeric library for Scala which is intended to be generic, fast, and precise.
import spire.syntax.cfor._

...


## Mode of operation

Here the user can define the mode of operation.
* **rdd_offline_mode**: If false it means the notebook will create all data from scratch and store grid0, grid0_index, protected_extent and num_cols_rows (from grid0) into HDFS. Otherwise, these data structures are read from HDFS.
* **matrix_offline_mode**: If false it means the notebook will create a mtrix,  transposed it and save it to HDFS. Otherwise, these data structures are read from HDFS.
* **kmeans_offline_mode**: If false it means the notebook will train kmeans and run kemans and store kmeans model into HDFS. Otherwise, these data structures are read from HDFS.

It is also possible to define which directory of GeoTiffs is to be used and on which **band** to run Kmeans. The options are
* **BloomFinal** or **LeafFinal** which are multi-band (**4 bands**)
* **DamageIndex** and **LastFreeze** which are single-band and if set band_num higher, it will reset to 0

For kmeans the user can define the **number of iterations** and **number of clusters** as an inclusive range. Such range is defined using **minClusters**, **maxClusters**, and **stepClusters**. These variables will set a loop starting at **minClusters** and stopping at **maxClusters** (inclusive), iterating **stepClusters** at the time. <span style="color:red">Note that when using a range **kemans offline mode** is not possible and it will be reset to **online mode**</span>.

### Mode of Operation setup
<a id='mode_of_operation_setup'></a>

In [2]:
//Operation mode
var rdd_offline_mode = true
var matrix_offline_mode = true
var kmeans_offline_mode = true

//GeoTiffs to be read from "hdfs:///user/hadoop/spring-index/"
var dir_path = "hdfs:///user/hadoop/spring-index/"
var offline_dir_path = "hdfs:///user/emma/spring-index/"
var geoTiff_dir = "LeafFinal"
var band_num = 3

//Years between (inclusive) 1980 - 2015
var model_first_year = 1989
var model_last_year = 2014

//Mask
val toBeMasked = true
val mask_path = "hdfs:///user/hadoop/usa_mask.tif"

//Kmeans number of iterations and clusters
var numIterations = 75
var minClusters = 210
var maxClusters = 500
var stepClusters = 10
var save_kmeans_model = false

rdd_offline_mode = true
matrix_offline_mode = true
kmeans_offline_mode = true
dir_path = hdfs:///user/hadoop/spring-index/
offline_dir_path = hdfs:///user/emma/spring-index/
geoTiff_dir = LeafFinal
band_num = 3
model_first_year = 1989
model_last_year = 2014
toBeMasked = true
mask_path = hdfs:///user/hadoop/usa_mask.tif
numIterations = 75
minClusters = 210
maxClusters = 500
stepClusters = 10
save_kmeans_model = false


false


<span style="color:red">DON'T MODIFY ANY PIECE OF CODE FROM HERE ON!!!</span>.


### Mode of operation validation

In [3]:
//Validation, do not modify these lines.
var single_band = false
if (geoTiff_dir == "BloomFinal" || geoTiff_dir == "LeafFinal") {
    single_band = false
} else if (geoTiff_dir == "LastFreeze" || geoTiff_dir == "DamageIndex") {
    single_band = true
    if (band_num > 0) {
        println("Since LastFreezze and DamageIndex are single band, we will use band 0!!!")
        band_num  = 0
    }
} else {
    println("Directory unknown, please set either BloomFinal, LeafFinal, LastFreeze or DamageIndex!!!")
}

if (minClusters > maxClusters) {
    maxClusters = minClusters
    stepClusters = 1
}
if (stepClusters < 1) {
    stepClusters = 1
}

//Paths to store data structures for Offline runs
var mask_str = ""
if (toBeMasked)
    mask_str = "_mask"
var grid0_path = offline_dir_path + geoTiff_dir + "/grid0" + "_"+ band_num + mask_str
var grid0_index_path = offline_dir_path + geoTiff_dir + "/grid0_index" + "_"+ band_num + mask_str
var grids_noNaN_path = offline_dir_path + geoTiff_dir + "/grids_noNaN" + "_"+ band_num + mask_str
var metadata_path = offline_dir_path + geoTiff_dir + "/metadata" + "_"+ band_num + mask_str
var grids_matrix_path = offline_dir_path + geoTiff_dir + "/grids_matrix" + "_"+ band_num + mask_str

//Check offline modes
var conf = sc.hadoopConfiguration
var fs = org.apache.hadoop.fs.FileSystem.get(conf)

val rdd_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(grid0_path))
val matrix_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(grids_matrix_path))
                                      
if (rdd_offline_mode != rdd_offline_exists) {
    println("\"Load GeoTiffs\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + rdd_offline_exists.toString())
    rdd_offline_mode = rdd_offline_exists
} 
if (matrix_offline_mode != matrix_offline_exists) {
    println("\"Matrix\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + matrix_offline_exists.toString())
    matrix_offline_mode = matrix_offline_exists
}

if (!fs.exists(new org.apache.hadoop.fs.Path(mask_path))) {
    println("The mask path: " + mask_path + " is invalid!!!")
}

//Years
val model_years = 1980 to 2015

if (!model_years.contains(model_first_year) || !(model_years.contains(model_last_year))) {
  println("Invalid range of years for " + geoTiff_dir + ". I should be between " + model_first_year + " and " + model_last_year)
  System.exit(0)
}

var model_years_range = (model_years.indexOf(model_first_year), model_years.indexOf(model_last_year))

var num_kmeans :Int  = 1
if (minClusters != maxClusters) {
    num_kmeans = ((maxClusters - minClusters) / stepClusters) + 1
}
println(num_kmeans)
var kmeans_model_paths :Array[String] = Array.fill[String](num_kmeans)("")
var wssse_path :String = offline_dir_path + geoTiff_dir + "/" + numIterations +"_wssse"
var geotiff_hdfs_paths :Array[String] = Array.fill[String](num_kmeans)("")
var geotiff_tmp_paths :Array[String] = Array.fill[String](num_kmeans)("")

if (num_kmeans > 1) {
    var numClusters_id = 0
    cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
        kmeans_model_paths(numClusters_id) = offline_dir_path + geoTiff_dir + "/kmeans_model_" + band_num + "_" + numClusters + "_" + numIterations
        
        //Check if the file exists
        val kmeans_exist = fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(numClusters_id)))
        if (kmeans_exist && !kmeans_offline_mode) {
            println("The kmeans model path " + kmeans_model_paths(numClusters_id) + " exists, please remove it.")
        } else if (!kmeans_exist && kmeans_offline_mode) {
            kmeans_offline_mode = false
        }
        
        geotiff_hdfs_paths(numClusters_id) = offline_dir_path + geoTiff_dir + "/clusters_" + band_num + "_" + numClusters + "_" + numIterations + ".tif"
        geotiff_tmp_paths(numClusters_id) = "/tmp/clusters_" + band_num + "_" + geoTiff_dir + "_" + numClusters + "_" + numIterations + ".tif"
        if (fs.exists(new org.apache.hadoop.fs.Path(geotiff_hdfs_paths(numClusters_id)))) {
            println("There is already a GeoTiff with the path: " + geotiff_hdfs_paths(numClusters_id) + ". Please make either a copy or move it to another location, otherwise, it will be over-written.")
        }
        numClusters_id += 1
    }
    kmeans_offline_mode = false
} else { 
    kmeans_model_paths(0) = offline_dir_path + geoTiff_dir + "/kmeans_model_" + band_num + "_" + minClusters + "_" + numIterations
    val kmeans_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(0)))
    if (kmeans_offline_mode != kmeans_offline_exists) {
        println("\"Kmeans\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + kmeans_offline_exists.toString())
        kmeans_offline_mode = kmeans_offline_exists
    }
    geotiff_hdfs_paths(0) = offline_dir_path + geoTiff_dir + "/clusters_" + band_num + "_" + minClusters + "_" + numIterations + ".tif"
    geotiff_tmp_paths(0) = "/tmp/clusters_" + band_num + "_" + geoTiff_dir + "_" + minClusters + "_" + numIterations + ".tif"
    if (fs.exists(new org.apache.hadoop.fs.Path(geotiff_hdfs_paths(0)))) {
        println("There is already a GeoTiff with the path: " + geotiff_hdfs_paths(0) + ". Please make either a copy or move it to another location, otherwise, it will be over-written.")
    }
}

Waiting for a Spark session to start...

30


single_band = false
mask_str = _mask
grid0_path = hdfs:///user/emma/spring-index/LeafFinal/grid0_mask
grid0_index_path = hdfs:///user/emma/spring-index/LeafFinal/grid0_index_mask
grids_noNaN_path = hdfs:///user/emma/spring-index/LeafFinal/grids_noNaN_mask
metadata_path = hdfs:///user/emma/spring-index/LeafFinal/metadata_mask
grids_matrix_path = hdfs:///user/emma/spring-index/LeafFinal/grids_matrix_mask
conf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml, file:/usr/lib/spark-2.1.1-bin-without-hadoop/conf/hive-site.xml
fs = DFS[DFSClient[clientName=DFSClient_N...


DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_2004373125_36, ugi=emma (auth:SIMPLE)]]

## Functions to (de)serialize any structure into Array[Byte]

In [4]:
def serialize(value: Any): Array[Byte] = {
    val out_stream: ByteArrayOutputStream = new ByteArrayOutputStream()
    val obj_out_stream = new ObjectOutputStream(out_stream)
    obj_out_stream.writeObject(value)
    obj_out_stream.close
    out_stream.toByteArray
}

def deserialize(bytes: Array[Byte]): Any = {
    val obj_in_stream = new ObjectInputStream(new ByteArrayInputStream(bytes))
    val value = obj_in_stream.readObject
    obj_in_stream.close
    value
}

serialize: (value: Any)Array[Byte]
deserialize: (bytes: Array[Byte])Any


## Load GeoTiffs

Using GeoTrellis all GeoTiffs of a directory will be loaded into a RDD. Using the RDD, we extract a grid from the first file to lated store the Kmeans cluster_IDS, we build an Index for populate such grid and we filter out here all NaN values.

In [5]:
val t0 = System.nanoTime()
//Global variables
var projected_extent = new ProjectedExtent(new Extent(0,0,0,0), CRS.fromName("EPSG:3857"))
var grid0: RDD[(Long, Double)] = sc.emptyRDD
var grid0_index: RDD[Long] = sc.emptyRDD
var grids_noNaN_RDD: RDD[Array[Double]] = sc.emptyRDD
var num_cols_rows :(Int, Int) = (0, 0)
var cellT :CellType = UByteCellType
var grids_RDD :RDD[Array[Double]] = sc.emptyRDD
var mask_tile0 :Tile = new SinglebandGeoTiff(geotrellis.raster.ArrayTile.empty(cellT, num_cols_rows._1, num_cols_rows._2), projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions.DEFAULT).tile
var grid_cells_size :Long = 0

//Load Mask
if (toBeMasked) {
    val mask_tiles_RDD = sc.hadoopGeoTiffRDD(mask_path).values
    val mask_tiles_withIndex = mask_tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
    mask_tile0 = (mask_tiles_withIndex.filter(m => m._1==0).values.collect())(0)
}

//Local variables
val pattern: String = "tif"
val filepath: String = dir_path + geoTiff_dir

if (rdd_offline_mode) {
    grids_noNaN_RDD = sc.objectFile(grids_noNaN_path)
    grid0 = sc.objectFile(grid0_path)
    grid0_index = sc.objectFile(grid0_index_path)

    val metadata = sc.sequenceFile(metadata_path, classOf[IntWritable], classOf[BytesWritable]).map(_._2.copyBytes()).collect()
    projected_extent = deserialize(metadata(0)).asInstanceOf[ProjectedExtent]
    num_cols_rows = (deserialize(metadata(1)).asInstanceOf[Int], deserialize(metadata(2)).asInstanceOf[Int])
    cellT = deserialize(metadata(3)).asInstanceOf[CellType]  
} else {
    if (single_band) {
        //Lets load a Singleband GeoTiffs and return RDD just with the tiles.
        var tiles_RDD :RDD[Tile] = sc.hadoopGeoTiffRDD(filepath, pattern).values
            
        //Retrive the numbre of cols and rows of the Tile's grid
        val tiles_withIndex = tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
        val tile0 = (tiles_withIndex.filter(m => m._1==0).values.collect())(0)
        num_cols_rows = (tile0.cols,tile0.rows)
        cellT = tile0.cellType
    
        if (toBeMasked) {
            val mask_tile_broad :Broadcast[Tile] = sc.broadcast(mask_tile0)
            grids_RDD = tiles_RDD.map(m => m.localInverseMask(mask_tile_broad.value, 1, 0).toArrayDouble())
        } else {
            grids_RDD = tiles_RDD.map(m => m.toArrayDouble())
        }
    } else {
        //Lets load Multiband GeoTiffs and return RDD just with the tiles.
        val tiles_RDD = sc.hadoopMultibandGeoTiffRDD(filepath, pattern).values
    
        //Retrive the numbre of cols and rows of the Tile's grid
        val tiles_withIndex = tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
        val tile0 = (tiles_withIndex.filter(m => m._1==0).values.collect())(0)
        num_cols_rows = (tile0.cols,tile0.rows)
        cellT = tile0.cellType
    
        //Lets read the average of the Spring-Index which is stored in the 4th band
        val band_numB :Broadcast[Int] = sc.broadcast(band_num)
        if (toBeMasked) {
            val mask_tile_broad :Broadcast[Tile] = sc.broadcast(mask_tile0)
            grids_RDD = tiles_RDD.map(m => m.band(band_numB.value).localInverseMask(mask_tile_broad.value, 1, -1000).toArrayDouble())
        } else {
            grids_RDD = tiles_RDD.map(m => m.band(band_numB.value).toArrayDouble())
        }
    }

    //Retrieve the ProjectExtent which contains metadata such as CRS and bounding box
    val projected_extents_withIndex = sc.hadoopGeoTiffRDD(filepath, pattern).keys.zipWithIndex().map{case (e,v) => (v,e)}
    projected_extent = (projected_extents_withIndex.filter(m => m._1 == 0).values.collect())(0)

    //Get Index for each Cell
    val grids_withIndex = grids_RDD.zipWithIndex().map { case (e, v) => (v, e) }
    
    if (toBeMasked) {
        grid0_index = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.filter(m => m._1 != -1000.0).map { case (v, i) => (i) }
    } else {
        //Dense vector
        grid0_index = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.filter(m => !m._1.isNaN).map { case (v, i) => (i) }
        //Sparse Vector
        //grid0_index = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.map { case (v, i) => (i) }
    }
    //Get the Tile's grid
    grid0 = grids_withIndex.filter(m => m._1 == 0).values.flatMap( m => m).zipWithIndex.map{case (v,i) => (i,v)}

    //Lets filter out NaN
    if (toBeMasked) {
        grids_noNaN_RDD = grids_RDD.map(m => m.filter(m => m != -1000.0))
    } else {
        //Dense Vector
        grids_noNaN_RDD = grids_RDD
        
        //Parse Vector
        //grids_noNaN_RDD = grids_RDD.map(m => m.filter(!_.isNaN))
    }
    
    //Store data in HDFS
    grid0.saveAsObjectFile(grid0_path)
    grid0_index.saveAsObjectFile(grid0_index_path)
    grids_noNaN_RDD.saveAsObjectFile(grids_noNaN_path)
    
    val grids_noNaN_RDD_withIndex = grids_noNaN_RDD.zipWithIndex().map { case (e, v) => (v, e) }
    grids_noNaN_RDD = grids_noNaN_RDD_withIndex.filterByRange(model_years_range._1, model_years_range._2).values
    
    val writer: SequenceFile.Writer = SequenceFile.createWriter(conf,
        Writer.file(metadata_path),
        Writer.keyClass(classOf[IntWritable]),
        Writer.valueClass(classOf[BytesWritable])
    )

    writer.append(new IntWritable(1), new BytesWritable(serialize(projected_extent)))
    writer.append(new IntWritable(2), new BytesWritable(serialize(num_cols_rows._1)))
    writer.append(new IntWritable(3), new BytesWritable(serialize(num_cols_rows._2)))
    writer.append(new IntWritable(4), new BytesWritable(serialize(cellT)))
    writer.hflush()
    writer.close()
}
grid_cells_size = grid0_index.count().toInt
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

Elapsed time: 7201378755ns


t0 = 47170647752218
projected_extent = ProjectedExtent(Extent(-126.30312894720473, 14.29219617034159, -56.162671563152486, 49.25462702827337),geotrellis.proj4.CRS$$anon$3@41d0d1b7)
grid0 = MapPartitionsRDD[14] at objectFile at <console>:103
grid0_index = MapPartitionsRDD[16] at objectFile at <console>:104
grids_noNaN_RDD = MapPartitionsRDD[12] at objectFile at <console>:102
num_cols_rows = (7808,3892)
cellT = uint8raw
grids_RDD = EmptyRDD[3] at emptyRDD at <console>:87
mask_tile0 = FloatRawArrayTile([F@5091905f,7808,3892)


pattern:...


FloatRawArrayTile([F@5091905f,7808,3892)

## Matrix

We need to do a Matrix transpose to have clusters per cell and not per year. With a GeoTiff representing a single year, the loaded data looks liks this:
```
bands_RDD.map(s => Vectors.dense(s)).cache()

//The vectors are rows and therefore the matrix will look like this:
[
Vectors.dense(0.0, 1.0, 2.0),
Vectors.dense(3.0, 4.0, 5.0),
Vectors.dense(6.0, 7.0, 8.0),
Vectors.dense(9.0, 0.0, 1.0)
]
```

To achieve that we convert the **RDD[Vector]** into a distributed Matrix, a [**CoordinateMatrix**](https://spark.apache.org/docs/latest/mllib-data-types.html#coordinatematrix), which as a **transpose** method.

In [6]:
val t0 = System.nanoTime()
//Global variables
var grids_matrix: RDD[Vector] = sc.emptyRDD
val grid_cells_sizeB = sc.broadcast(grid_cells_size)

if (matrix_offline_mode) {
    grids_matrix = sc.objectFile(grids_matrix_path)
} else {
    //Dense Vector
    val mat :RowMatrix = new RowMatrix(grids_noNaN_RDD.map(m => Vectors.dense(m)))
    //Sparse Vector
    val mat :RowMatrix = new RowMatrix(grids_noNaN_RDD.map(m => m.zipWithIndex).map(m => m.filter(!_._1.isNaN)).map(m => Vectors.sparse(grid_cells_sizeB.value.toInt, m.map(v => v._2), m.map(v => v._1))))
    // Split the matrix into one number per line.
    val byColumnAndRow = mat.rows.zipWithIndex.map {
        case (row, rowIndex) => row.toArray.zipWithIndex.map {
            case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
        }   
    }.flatMap(x => x)
    
    val matt: CoordinateMatrix = new CoordinateMatrix(byColumnAndRow)
    val matt_T = matt.transpose()
    //grids_matrix = matt_T.toRowMatrix().rows
    grids_matrix = matt_T.toIndexedRowMatrix().rows.sortBy(_.index).map(_.vector)
    grids_matrix.saveAsObjectFile(grids_matrix_path)
}
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

Elapsed time: 18307634ns


t0 = 47180808240332
grids_matrix = MapPartitionsRDD[21] at objectFile at <console>:72
t1 = 47180826547966


47180826547966

## Kmeans

We use Kmeans from Sparl-MLlib. The user should only modify the variables on Kmeans setup.

### Kmeans Training

In [7]:
val t0 = System.nanoTime()
//Global variables
var kmeans_models :Array[KMeansModel] = new Array[KMeansModel](num_kmeans)
var wssse_data :List[(Int, Int, Double)] = List.empty

if (kmeans_offline_mode) {
    var numClusters_id = 0
    cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
        if (!fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(numClusters_id)))) {
            println("One of the files does not exist, we will abort!!!")
            System.exit(0)
        } else {
            kmeans_models(numClusters_id) = KMeansModel.load(sc, kmeans_model_paths(numClusters_id))
        }
        numClusters_id += 1
    }
    val wssse_data_RDD :RDD[(Int, Int, Double)]  = sc.objectFile(wssse_path)
    wssse_data  = wssse_data_RDD.collect().toList
} else {
    var numClusters_id = 0
    if (fs.exists(new org.apache.hadoop.fs.Path(wssse_path))) {
        val wssse_data_RDD :RDD[(Int, Int, Double)]  = sc.objectFile(wssse_path)
        wssse_data  = wssse_data_RDD.collect().toList
    }
    grids_matrix.cache()
    cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
        println(numClusters)
        kmeans_models(numClusters_id) = {
            KMeans.train(grids_matrix, numClusters, numIterations)
        }

        // Evaluate clustering by computing Within Set Sum of Squared Errors
        val WSSSE = kmeans_models(numClusters_id).computeCost(grids_matrix)
        println("Within Set Sum of Squared Errors = " + WSSSE)
                
        wssse_data = wssse_data :+ (numClusters, numIterations, WSSSE)
        
        //Save kmeans model
        if (save_kmeans_model) {
            if (!fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(numClusters_id)))) {
                kmeans_models(numClusters_id).save(sc, kmeans_model_paths(numClusters_id))
            }
        }
        numClusters_id += 1
            
        if (fs.exists(new org.apache.hadoop.fs.Path(wssse_path))) {
            println("We will delete the wssse file")
            try { fs.delete(new org.apache.hadoop.fs.Path(wssse_path), true) } catch { case _ : Throwable => { } }
        }
    
        println("Lets create it with the new data")
        sc.parallelize(wssse_data, 1).saveAsObjectFile(wssse_path)
    }

    //Un-persist it to save memory
    grids_matrix.unpersist()

}
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

210
Within Set Sum of Squared Errors = 3.3919045694046974E9                         
220
Within Set Sum of Squared Errors = 3.327929298789376E9                          
230
Within Set Sum of Squared Errors = 3.270508461779917E9                          
240
Within Set Sum of Squared Errors = 3.210822688087481E9                          
250
Within Set Sum of Squared Errors = 3.1630201462281303E9                         
260
Within Set Sum of Squared Errors = 3.107648131494194E9                          
270
Within Set Sum of Squared Errors = 3.0598154144266157E9                         
280
Within Set Sum of Squared Errors = 3.015164080011255E9                          
290
Within Set Sum of Squared Errors = 2.9765264838213587E9                         
300
Within Set Sum of Squared Errors = 2.9105114866128073E9                         
310
Within Set Sum of Squared Errors = 2.8914331039926867E9                         
320
Within Set Sum of Squared Errors = 2.856370848402045E9       

t0 = 47182120153945
kmeans_models = Array(org.apache.spark.mllib.clustering.KMeansModel@4cb35692, org.apache.spark.mllib.clustering.KMeansModel@1d83f314, org.apache.spark.mllib.clustering.KMeansModel@7c6b5996, org.apache.spark.mllib.clustering.KMeansModel@b597412, org.apache.spark.mllib.clustering.KMeansModel@5d88a42d, org.apache.spark.mllib.clustering.KMeansModel@5f91e7db, org.apache.spark.mllib.clustering.KMeansModel@7990a256, org.apache.spark.mllib.clustering.KMeansModel@3929a092, org.apache.spark.mllib.clustering.KMeansModel@2198d256, org.apache.spark.mllib.clustering.KMeansModel@71e75681, org.apache.spark.mllib.clustering.KMeansModel@d19c8f9, org.apache.spark.mllib.clustering.KMeansModel@79c0d4f8, org.apache.spark.mllib.cl...


[org.apache.spark.mllib.clustering.KMeansModel@4cb35692, org.apache.spark.mllib.clustering.KMeansModel@1d83f314, org.apache.spark.mllib.clustering.KMeansModel@7c6b5996, org.apache.spark.mllib.clustering.KMeansModel@b597412, org.apache.spark.mllib.clustering.KMeansModel@5d88a42d, org.apache.spark.mllib.clustering.KMeansModel@5f91e7db, org.apache.spark.mllib.clustering.KMeansModel@7990a256, org.apache.spark.mllib.clustering.KMeansModel@3929a092, org.apache.spark.mllib.clustering.KMeansModel@2198d256, org.apache.spark.mllib.clustering.KMeansModel@71e75681, org.apache.spark.mllib.clustering.KMeansModel@d19c8f9, org.apache.spark.mllib.clustering.KMeansModel@79c0d4f8, org.apache.spark.mllib.clustering.KMeansModel@1eada3f4, org.apache.spark.mllib.clustering.KMeansModel@55b5bdfe, org.apache.spark.mllib.clustering.KMeansModel@1628f9b1, org.apache.spark.mllib.clustering.KMeansModel@69b08898, org.apache.spark.mllib.clustering.KMeansModel@1af12459, org.apache.spark.mllib.clustering.KMeansModel@62d

### Inspect WSSSE

In [8]:
val t0 = System.nanoTime()
//current
println(wssse_data)

//from disk
if (fs.exists(new org.apache.hadoop.fs.Path(wssse_path))) {
    var wssse_data_tmp :RDD[(Int, Int, Double)] = sc.objectFile(wssse_path)//.collect()//.toList
    println(wssse_data_tmp.collect().toList)    
}
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

List((10,75,1.5833035912018782E10), (20,75,1.0537893613199553E10), (30,75,8.477877923679263E9), (40,75,7.379593626293102E9), (50,75,6.610350490917555E9), (60,75,6.090931317732018E9), (70,75,5.655089899173892E9), (80,75,5.296610424032334E9), (90,75,4.993140085788995E9), (100,75,4.762312237508368E9), (110,75,4.562118552615672E9), (120,75,4.367716817142325E9), (130,75,4.22209182009944E9), (140,75,4.077189067356344E9), (150,75,3.9582148213718586E9), (160,75,3.82295262984698E9), (170,75,3.7260327762321296E9), (180,75,3.6493610548466563E9), (190,75,3.560497470770442E9), (200,75,3.4810389079324985E9), (210,75,3.3919045694046974E9), (220,75,3.327929298789376E9), (230,75,3.270508461779917E9), (240,75,3.210822688087481E9), (250,75,3.1630201462281303E9), (260,75,3.107648131494194E9), (270,75,3.0598154144266157E9), (280,75,3.015164080011255E9), (290,75,2.9765264838213587E9), (300,75,2.9105114866128073E9), (310,75,2.8914331039926867E9), (320,75,2.856370848402045E9), (330,75,2.809690702179724E9), (3

t0 = 87566499491230
t1 = 87568571647773


87568571647773

### Run Kmeans clustering

Run Kmeans and obtain the clusters per each cell.

In [9]:
val t0 = System.nanoTime()
//Cache it so kmeans is more efficient
grids_matrix.cache()

var kmeans_res: Array[RDD[Int]] = Array.fill(num_kmeans)(sc.emptyRDD)
var numClusters_id = 0
cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
    kmeans_res(numClusters_id) = kmeans_models(numClusters_id).predict(grids_matrix)
    numClusters_id += 1
}

//Un-persist it to save memory
grids_matrix.unpersist()
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

Elapsed time: 23728787557ns


t0 = 87575967233119
kmeans_res = Array(MapPartitionsRDD[5099] at map at KMeansModel.scala:69, MapPartitionsRDD[5100] at map at KMeansModel.scala:69, MapPartitionsRDD[5101] at map at KMeansModel.scala:69, MapPartitionsRDD[5102] at map at KMeansModel.scala:69, MapPartitionsRDD[5103] at map at KMeansModel.scala:69, MapPartitionsRDD[5104] at map at KMeansModel.scala:69, MapPartitionsRDD[5105] at map at KMeansModel.scala:69, MapPartitionsRDD[5106] at map at KMeansModel.scala:69, MapPartitionsRDD[5107] at map at KMeansModel.scala:69, MapPartitionsRDD[5108] at map at KMeansModel.scala:69, MapPartitionsRDD[5109] at map at KMeansModel.scala:69, MapPartitionsRDD[5110] at map at KMeansModel.scala:69, MapPartitionsRDD[5111] at map at KMeansModel.scala:69,...


[MapPartitionsRDD[5099] at map at KMeansModel.scala:69, MapPartitionsRDD[5100] at map at KMeansModel.scala:69, MapPartitionsRDD[5101] at map at KMeansModel.scala:69, MapPartitionsRDD[5102] at map at KMeansModel.scala:69, MapPartitionsRDD[5103] at map at KMeansModel.scala:69, MapPartitionsRDD[5104] at map at KMeansModel.scala:69, MapPartitionsRDD[5105] at map at KMeansModel.scala:69, MapPartitionsRDD[5106] at map at KMeansModel.scala:69, MapPartitionsRDD[5107] at map at KMeansModel.scala:69, MapPartitionsRDD[5108] at map at KMeansModel.scala:69, MapPartitionsRDD[5109] at map at KMeansModel.scala:69, MapPartitionsRDD[5110] at map at KMeansModel.scala:69, MapPartitionsRDD[5111] at map at KMeansModel.scala:69, MapPartitionsRDD[5112] at map at KMeansModel.scala:69, MapPartitionsRDD[5113] at map at KMeansModel.scala:69, MapPartitionsRDD[5114] at map at KMeansModel.scala:69, MapPartitionsRDD[5115] at map at KMeansModel.scala:69, MapPartitionsRDD[5116] at map at KMeansModel.scala:69, MapPartit

#### Sanity test

It can be skipped, it only shows the cluster ID for the first 50 cells

In [10]:
val t0 = System.nanoTime()
val kmeans_res_out = kmeans_res(0).take(150)
kmeans_res_out.foreach(print)

println(kmeans_res_out.size)
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535353535150
Elapsed time: 1156923550ns


t0 = 87606668472749
kmeans_res_out = Array(35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35)
t1 = 87607825396299


87607825396299

## Build GeoTiff with Kmeans cluster_IDs

The Grid with the cluster IDs is stored in a SingleBand GeoTiff and uploaded to HDFS.

### Assign cluster ID to each grid cell and save the grid as SingleBand GeoTiff

To assign the clusterID to each grid cell it is necessary to get the indices of gird cells they belong to. The process is not straight forward because the ArrayDouble used for the creation of each dense Vector does not contain the NaN values, therefore there is not a direct between the indices in the Tile's grid and the ones in **kmeans_res** (kmeans result).

To join the two RDDS the knowledge was obtaing from a stackoverflow post on [how to perform basic joins of two rdd tables in spark using python](https://stackoverflow.com/questions/31257077/how-do-you-perform-basic-joins-of-two-rdd-tables-in-spark-using-python).

In [ ]:
val t0 = System.nanoTime()
var numClusters_id = 0
val grid0_index_I = grid0_index.zipWithIndex().map{ case (v,i) => (i,v)}
cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
    //Merge two RDDs, one containing the clusters_ID indices and the other one the indices of a Tile's grid cells
    val cluster_cell_pos = ((kmeans_res(numClusters_id).zipWithIndex().map{ case (v,i) => (i,v)}).join(grid0_index_I).map{ case (k,(v,i)) => (v,i)}

    //Associate a Cluster_IDs to respective Grid_cell
    val grid_clusters = grid0.leftOuterJoin(cluster_cell_pos.map{ case (c,i) => (i.toLong, c)})

    //Convert all None to NaN
    val grid_clusters_res = grid_clusters.sortByKey(true).map{case (k, (v, c)) => if (c == None) (k, Double.NaN) else (k, c.get.toDouble)}
    
    //Define a Tile
    val cluster_cells :Array[Double] = grid_clusters_res.values.collect()
    val cluster_cellsD = DoubleArrayTile(cluster_cells, num_cols_rows._1, num_cols_rows._2)
    val geoTif = new SinglebandGeoTiff(cluster_cellsD, projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions(compression.DeflateCompression))

    //Save to /tmp/
    GeoTiffWriter.write(geoTif, geotiff_tmp_paths(numClusters_id))

    //Upload to HDFS
    var cmd = "hadoop dfs -copyFromLocal -f " + geotiff_tmp_paths(numClusters_id) + " " + geotiff_hdfs_paths(numClusters_id)
    Process(cmd)!
    
    //Remove from /tmp/
    cmd = "rm -fr " + geotiff_tmp_paths(numClusters_id)
    Process(cmd)!
    
    numClusters_id += 1
}
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

[Stage 4788:====================================================> (31 + 1) / 32]DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

[Stage 4801:====================================================> (31 + 1) / 32]

# [Visualize results](plot_kmeans_clusters.ipynb) --------------- [Plot WSSE](kmeans_wsse.ipynb)